In [1]:
import numpy as np
import pandas as pd
import re
import collections
import operator
import simpy
import random

In [2]:
def estatistica_descritiva(data,campo):
    print(f"{campo} Media :",data[campo].mean())
    print(f"{campo} Mediana",data[campo].median())
    print(f"{campo} Moda",list(data[campo].mode()))
    print(f"{campo} Minimo",data[campo].min())
    print(f"{campo} Maximo",data[campo].max())
    print(f"{campo} Amplitude",data[campo].max()-data[campo].min())
    print(f"{campo} Desvio padrão",data[campo].std())
    print(f"{campo} Variacia",data[campo].var())
    print(f"{campo} Coeficinte de variação",data[campo].std()/data[campo].mean())
    print(f"{campo} Assimetria",data[campo].skew())

In [54]:
def tempo_medio (data,trantime=False,breaktime=False):
    
    if trantime:
        produtos_segundos = 0
        for index, trasacao in data.iterrows():
            produtos_segundos += trasacao["TranTime"]/trasacao['ArtNum']
        media_produtos_segundos = produtos_segundos/len(data.index)
        return(media_produtos_segundos)
    
    elif breaktime:
        produtos_segundos = 0
        for index, trasacao in data.iterrows():
            produtos_segundos += (trasacao["BreakTime"])/trasacao['ArtNum']
        media_produtos_segundos=produtos_segundos/len(data.index)
        return(media_produtos_segundos)
        
    else:
        produtos_segundos = 0
        for index, trasacao in data.iterrows():
            produtos_segundos +=np.add(trasacao["TranTime"],trasacao["BreakTime"])/trasacao['ArtNum']
        media_produtos_segundos=produtos_segundos/len(data.index)
        return(media_produtos_segundos)

In [4]:
trasacao = pd.read_csv("data/POS_transactions_20190328-20190410.csv")

In [5]:
trasacao.dtypes

WorkstationGroupID      int64
TranID                float64
BeginDateTime          object
EndDateTime            object
OperatorID              int64
TranTime                int64
BreakTime               int64
ArtNum                  int64
TNcash                   bool
TNcard                   bool
Amount                float64
dtype: object

In [6]:
trasacao_workstation_service = trasacao.query('WorkstationGroupID == 1')

In [7]:
trasacao_workstation_service_date = trasacao_workstation_service.loc[trasacao_workstation_service.BeginDateTime.str.contains('2019-04-03T10|2019-04-03T11|2019-04-03T12', flags=re.I, regex=True)]

## Removendo as linha com TranTime igual a zero

In [83]:
for index, transacao in trasacao_workstation_service_date.iterrows():
    if transacao["TranTime"]==0:
        trasacao_workstation_service_date=trasacao_workstation_service_date.drop([index])

In [85]:
trasacao_workstation_service_date.describe()

,WorkstationGroupID,TranID,OperatorID,TranTime,BreakTime,ArtNum,Amount
count,510.0,5.100000e+02,510.000000,510.000000,510.000000,510.000000,510.000000
mean,1.0,6.874224e+13,220.229412,50.854902,33.194118,15.147059,62.173235
std,0.0,7.988460e+13,94.311403,40.053924,71.336329,14.968990,78.679271
min,1.0,1.904031e+11,105.000000,4.000000,6.000000,1.000000,2.170000
25%,1.0,1.904031e+13,128.000000,24.000000,15.000000,6.000000,21.337500
50%,1.0,1.904031e+13,301.000000,41.000000,21.000000,11.000000,41.380000
75%,1.0,1.904031e+14,311.000000,67.000000,32.000000,19.000000,72.785000
max,1.0,1.904031e+14,361.000000,309.000000,1147.000000,123.000000,811.750000


## Estatistica descritiva todas as formas de pagamentos

### Estatistica descritiva campo TranTime

In [9]:
estatistica_descritiva(trasacao_workstation_service_date,"TranTime")

TranTime Media : 50.85490196078431
TranTime Mediana 41.0
TranTime Moda [26, 31]
TranTime Minimo 4
TranTime Maximo 309
TranTime Amplitude 305
TranTime Desvio padrão 40.05392393613793
TranTime Variacia 1604.3168226819225
TranTime Coeficinte de variação 0.7876118602494735
TranTime Assimetria 2.0457119278440214


### Estatistica descritiva campo Breaktime

In [10]:
estatistica_descritiva(trasacao_workstation_service_date,"BreakTime")


BreakTime Media : 33.194117647058825
BreakTime Mediana 21.0
BreakTime Moda [17]
BreakTime Minimo 6
BreakTime Maximo 1147
BreakTime Amplitude 1141
BreakTime Desvio padrão 71.33632925114233
BreakTime Variacia 5088.871871027384
BreakTime Coeficinte de variação 2.1490653859107205
BreakTime Assimetria 12.041105864454947


### Estatistica Descritiva campo numeros de artigos

In [11]:
estatistica_descritiva(trasacao_workstation_service_date,"ArtNum")

ArtNum Media : 15.147058823529411
ArtNum Mediana 11.0
ArtNum Moda [3]
ArtNum Minimo 1
ArtNum Maximo 123
ArtNum Amplitude 122
ArtNum Desvio padrão 14.968990250918498
ArtNum Variacia 224.07066913209303
ArtNum Coeficinte de variação 0.9882440165654931
ArtNum Assimetria 2.6869050219189594


### Estatistica Descritiva campo TNcash e TNcard

In [12]:
trasacao_workstation_service_date["TNcash"].describe()

count      510
unique       2
top       True
freq       335
Name: TNcash, dtype: object

### Calculando o tempo medio de item por segundo



In [55]:
tempo_medio(trasacao_workstation_service_date)

9.205502027971132

###  Calculando o tempo medio de item por segundo considerando somente o tempo de trasação

In [86]:
tempo_medio(trasacao_workstation_service_date,trantime=True)

4.669640765279946

###  Calculando o tempo medio de item por segundo considerando somente o tempo de empacotamento

In [15]:
tempo_medio(trasacao_workstation_service_date,breaktime=True)

4.53586126269119


## Estatistica descritiva considerando apenas compras no dinheiro


In [16]:
trasacao_workstation_service_date_tncash = trasacao_workstation_service_date.query('TNcash == True')

### Estatistica descritiva campo TranTime

In [17]:
estatistica_descritiva(trasacao_workstation_service_date_tncash,"TranTime")

TranTime Media : 44.208955223880594
TranTime Mediana 35.0
TranTime Moda [11, 26, 28]
TranTime Minimo 4
TranTime Maximo 309
TranTime Amplitude 305
TranTime Desvio padrão 37.53096019896204
TranTime Variacia 1408.5729734560728
TranTime Coeficinte de variação 0.8489447445410051
TranTime Assimetria 2.6301824423971367


### Estatistica descritiva campo BreakTime

In [18]:
estatistica_descritiva(trasacao_workstation_service_date_tncash,"BreakTime")

BreakTime Media : 33.56417910447761
BreakTime Mediana 22.0
BreakTime Moda [17, 19]
BreakTime Minimo 8
BreakTime Maximo 940
BreakTime Amplitude 932
BreakTime Desvio padrão 57.94443885705379
BreakTime Variacia 3357.5579944588453
BreakTime Coeficinte de variação 1.7263773583344912
BreakTime Assimetria 12.235668470386164


### Estatistica descritiva campo ArtNum

In [19]:
estatistica_descritiva(trasacao_workstation_service_date_tncash,"ArtNum")

ArtNum Media : 13.170149253731344
ArtNum Mediana 10.0
ArtNum Moda [3]
ArtNum Minimo 1
ArtNum Maximo 123
ArtNum Amplitude 122
ArtNum Desvio padrão 13.458663871692076
ArtNum Variacia 181.13563321118957
ArtNum Coeficinte de variação 1.0219067082993756
ArtNum Assimetria 3.526173671756291


### Calculando o tempo medio de item por segundo

In [20]:
tempo_medio(trasacao_workstation_service_date_tncash)

9.980679204243227


###  Calculando o tempo medio de item por segundo considerando somente o tempo de trasação

In [21]:
tempo_medio(trasacao_workstation_service_date_tncash,trantime=True)

4.537310756315868


###  Calculando o tempo medio de item por segundo considerando somente o tempo de empacotamento

In [22]:
tempo_medio(trasacao_workstation_service_date_tncash,breaktime=True)

5.4433684479273605


## Estatistica descritiva considerando apenas no cartão de credito

In [23]:
trasacao_workstation_service_date_tncard = trasacao_workstation_service_date.query('TNcard == True')

### Estatistica descritiva campo TranTime

In [24]:
estatistica_descritiva(trasacao_workstation_service_date_tncard,"TranTime")

TranTime Media : 64.33714285714285
TranTime Mediana 52.0
TranTime Moda [31]
TranTime Minimo 13
TranTime Maximo 251
TranTime Amplitude 238
TranTime Desvio padrão 41.99487575277451
TranTime Variacia 1763.5695894909688
TranTime Coeficinte de variação 0.6527314376708002
TranTime Assimetria 1.3877977720260803


### Estatistica descritiva campo BreakTime

In [25]:
estatistica_descritiva(trasacao_workstation_service_date_tncard,"BreakTime")

BreakTime Media : 32.48
BreakTime Mediana 16.0
BreakTime Moda [14]
BreakTime Minimo 6
BreakTime Maximo 1147
BreakTime Amplitude 1141
BreakTime Desvio padrão 91.86445787511983
BreakTime Variacia 8439.078620689665
BreakTime Coeficinte de variação 2.828339220293098
BreakTime Assimetria 10.706897789257468


### Estatistica descritiva campo ArtNum

In [26]:
estatistica_descritiva(trasacao_workstation_service_date_tncard,"ArtNum")

ArtNum Media : 18.925714285714285
ArtNum Mediana 13.0
ArtNum Moda [3]
ArtNum Minimo 1
ArtNum Maximo 98
ArtNum Amplitude 97
ArtNum Desvio padrão 17.016225233061128
ArtNum Variacia 289.5519211822662
ArtNum Coeficinte de variação 0.8991061038000294
ArtNum Assimetria 1.7948384626166136


### Calculando o tempo medio de item por segundo

In [27]:
tempo_medio(trasacao_workstation_service_date_tncard)

7.877840225663656


### Calculando o tempo medio de item por transação

In [28]:
tempo_medio(trasacao_workstation_service_date_tncard,trantime=True)

5.0663667686371365


### Calculando o tempo medio item por tempo de empacotamento

In [29]:
tempo_medio(trasacao_workstation_service_date_tncard,breaktime=True)

2.8114734570265214


In [30]:
frequencia = collections.Counter(trasacao_workstation_service_date["ArtNum"])

In [31]:
frequencia_ordenada = sorted(frequencia.items(), key=operator.itemgetter(1), reverse=True)

In [32]:
frequencia_porcentagem = {}

In [33]:
frequencia_f = []
for frequencia in frequencia_ordenada:
    porcentagem = (frequencia[1]/len(trasacao_workstation_service_date.index))*100
    frequencia_f.append((frequencia[0],porcentagem))


In [34]:
frequencia_dividida_tamanho = {"qtd_pequena": [], "qtd_media": [], "qtd_grande": []}
soma_frequencia_pequena = 0
soma_frequencia_media = 0
soma_frequencia_grande = 0
for frequencia in frequencia_f:
    if frequencia[0] <= 10 :
        frequencia_dividida_tamanho["qtd_pequena"].append(frequencia[0])
        soma_frequencia_pequena += frequencia[1] 
    elif (frequencia[0] > 10) & (frequencia[0]<=30):
        frequencia_dividida_tamanho["qtd_media"].append(frequencia[0])
        soma_frequencia_media += frequencia[1]
    else :
        frequencia_dividida_tamanho["qtd_grande"].append(frequencia[0])
        soma_frequencia_grande += frequencia[1]
frequencia_dividida_tamanho["qtd_pequena"] = (soma_frequencia_pequena)
frequencia_dividida_tamanho["qtd_media"] = (soma_frequencia_media)
frequencia_dividida_tamanho["qtd_grande"] = (frequencia_dividida_tamanho["qtd_grande"],soma_frequencia_grande)

In [35]:
frequencia_dividida_tamanho["qtd_pequena"]

49.2156862745098

In [36]:
frequencia_dividida_tamanho["qtd_media"]

39.607843137254896

In [37]:
frequencia_dividida_tamanho["qtd_grande"]

([37,
  42,
  31,
  40,
  48,
  33,
  41,
  44,
  39,
  35,
  34,
  38,
  36,
  47,
  62,
  58,
  67,
  59,
  32,
  98,
  82,
  45,
  123,
  53,
  87,
  49,
  76,
  51,
  46,
  64,
  70,
  72,
  84],
 11.176470588235286)

## Calculando o tempo medio de item por segundo

In [38]:
OperatorIDs=trasacao_workstation_service_date[['OperatorID']]

In [39]:
operadores_unicos = OperatorIDs.drop_duplicates()
operadores_list = operadores_unicos.values

In [40]:
operadores = {}
for operador in operadores_list:
    operadores[int(operador)] = trasacao_workstation_service_date.query(f"OperatorID=={operador}")


# simulação simpy

In [41]:
class cliente:
  def __init__(self,id,n_itens):
    self.id_cliente = id
    self.n_itens = n_itens 
    self.tempo_inicial = None
    self.tempo_final = None
  
  
  def set_tempo_incial(self,tempo_incial):
    self.tempo_incial = tempo_inicial
  
  
  def set_tempo_final(self,tempo_final):
    self.tempo_final = tempo_final

In [42]:
class itens:
  def __init__(self):
    self.clientes=[]
    for i in range(1,80):
      self.clientes.append(random.randrange(1,20))
    for i in range(80,101):
      self.clientes.append(random.randrange(20,100))
Itens = itens()


In [43]:
def criação_de_filas_linear():
  index=0
  filas=[[],[],[],[]]
  clientes=[]
  #print(clientes)
  for cliente in Itens.clientes:
    if index == 4:
      index=0
    filas[index].append((cliente))
    index+=1
  for i in filas:
    print ("fila->",i,"tamanho da fila->",len(i))
  return (filas)

In [44]:
def criação_de_filas_com_prioridade():
  index=0
  filas=[[],[],[],[]]
  for cliente in Itens.clientes:
    if cliente <= 10:
      filas[0].append(cliente)
    if cliente >= 11 and cliente <= 30:
      filas[1].append(cliente)  
    if cliente >= 31 and cliente <=60:
      filas[2].append(cliente)
    if cliente > 60:
      filas[3].append(cliente)
  for i in filas:
    print ("fila->",i,"tamanho da fila->",len(i))
  return (filas)


In [45]:
def menor_peso_fila(filas):
    menor_peso=9999999999
    index_vez=0
    index=0
    for i in filas:
      if sum(i)>0:
        if (sum(i)) < menor_peso :
          menor_peso=sum(i)
          index_vez=index
        index+=1
      else:
        index_vez=index
        return (index_vez)
        
    return (index_vez)

In [46]:
def criação_de_filas():
  index=0
  filas=[[],[],[],[]]
  #print(clientes)
  for cliente in Itens.clientes:
    vez=(menor_peso_fila(filas))
    #print(index)
    filas[vez].append((cliente))
    index+=1
  for i in filas:
    print ("fila->",i,"tamanho da fila->",len(i))
  return (filas)#cada index representa um caixa 

In [47]:

class caixa:
  def __init__(self,numero_caixa,env,fila):
    self.numero_caixa = numero_caixa
    self.env = env
    self.fila = fila
    self.caixas=[]
    self.caixa_disponivel = self.env.event()
    self.env.process(self.caixa_aberto())
    self.env.process(self.transacao())
    self.clientes = []
    self.clientes_finalizados = []
    
    
  def set_embalador(self):
    self.embalador = True
  
  def set_caixas(self,caixas):
    self.caixas=caixas


  def transacao(self):
    index=0
    yield   self.caixa_disponivel
    #print('eu executo--->',len(self.clientes))
    Clientes=self.clientes.copy()
    #print(Clientes)
    for cliente_atual in Clientes:
      n_itens=cliente_atual.n_itens
      if len(self.clientes)>0:
        print("inicia cliente ##########################################"
              ,"Cliente atual",cliente_atual.id_cliente,"QTD iten do cliente",
              cliente_atual.n_itens,"Caixa",self.numero_caixa,"Tempo atual",self.env.now )
        #print('eu executo--> numero de itens',cliente_atual.n_itens)
        cliente_atual.tempo_inicial = self.env.now
        #print("o cliente  ",cliente_atual.id_cliente ,"está sendo atendido no tempo ", self.env.now,"caixa ", self.numero_caixa,'\n\n\#####################')
        while cliente_atual.n_itens > 0 :
          cliente_atual.n_itens = cliente_atual.n_itens-1
          yield   self.env.timeout( 1 )#tempo referente a trasação
        print("o cliente Saiu############################################### "," o cliente foi atendido o caixa",self.numero_caixa ,"está vazio","tempo atual",self.env.now,"\n\n\n\n")
        cliente_atual.set_tempo_final(self.env.now)
        cliente_atual.n_itens=n_itens
        self.clientes_finalizados.append(cliente_atual)
        #print('------------->id',cliente_atual.id_cliente)
        #print('cliente atual --->',cliente_atual.id_cliente)
        self.clientes.remove(cliente_atual)
    print("=========<>==================Acabou a fila do caixa ",self.numero_caixa,"\n\n\n\n")
    self.env.process(self.busca_cliente_outras_filas())



  
  
  def caixa_aberto(self):
    self.criar_fila()
    yield   self.env.timeout(2)
    self.caixa_disponivel.succeed()



  def busca_cliente_outras_filas(self):
      for caixa in self.caixas:
        
        if len(caixa.clientes)>1:
          print("caixa",self.numero_caixa,'--------------->>>Procurando cliente em outra fila, buscando no caixa',caixa.numero_caixa,"tamanho da fila outro caixa",len(caixa.clientes),self.env.now,"\n\n\n")
          self.clientes.append(caixa.clientes[-1])
          #print(caixa.clientes[-1].id_cliente,caixa.numero_caixa,"tamanho da fila",len(caixa.clientes))
          caixa.clientes.remove(caixa.clientes[-1])
          yield   self.env.timeout(10)
          self.env.process(self.transacao())
          break


  
  def criar_fila(self):
    index=0
    for i in self.fila:
      #print(i)
      c=cliente(index,i)
      self.clientes.append(c)
      index+=1
  
  def media_tempo_caixa(self):
    soma=0
    for cliente in self.clientes_finalizados:
      soma=soma+(cliente.tempo_final-cliente.tempo_inicial)
    return (soma/len(self.clientes_finalizados))

## simulações considerando o tempo de transação e embalagem

### simulação considerando pesos na fila

In [82]:
def main_fila_com_peso():
  clientes=[]
  env = simpy.Environment()
  filas = criação_de_filas()
  caixa1=caixa(1,env,filas[0])
  caixa2=caixa(2,env,filas[1])
  caixa3=caixa(3,env,filas[2])
  caixa4=caixa(4,env,filas[3])
  caixa1.set_caixas([caixa2,caixa3,caixa4])
  caixa2.set_caixas([caixa1,caixa3,caixa4])
  caixa3.set_caixas([caixa1,caixa2,caixa4])
  caixa4.set_caixas([caixa1,caixa2,caixa3])
  env.run()
  print("Caixa 1 media de tempo",round((caixa1.media_tempo_caixa()*tempo_medio(trasacao_workstation_service_date))/60,2),"minutos","Cliente atendidos",len(caixa1.clientes_finalizados),"Tempo de simulação caixa 1",round((caixa1.clientes_finalizados[-1].tempo_final*tempo_medio(trasacao_workstation_service_date))/60,2), "minutos")
  print("Caixa 2 media de tempo",round((caixa2.media_tempo_caixa()*tempo_medio(trasacao_workstation_service_date))/60,2),"minutos","Cliente atendidos",len(caixa2.clientes_finalizados),"Tempo de simulação caixa 2",round((caixa2.clientes_finalizados[-1].tempo_final*tempo_medio(trasacao_workstation_service_date))/60,2), "minutos")
  print("Caixa 3 media de tempo",round((caixa3.media_tempo_caixa()*tempo_medio(trasacao_workstation_service_date))/60,2),"minutos","Cliente atendidos",len(caixa3.clientes_finalizados),"Tempo de simulação caixa 3",round((caixa3.clientes_finalizados[-1].tempo_final*tempo_medio(trasacao_workstation_service_date))/60,2), "minutos")
  print("Caixa 4 media de tempo",round((caixa4.media_tempo_caixa()*tempo_medio(trasacao_workstation_service_date))/60,2),"minutos","Cliente atendidos",len(caixa4.clientes_finalizados),"Tempo de simulação caixa 4",round((caixa4.clientes_finalizados[-1].tempo_final*tempo_medio(trasacao_workstation_service_date))/60,2), "minutos")
  print("media de tempo dos caixas",round((((caixa1.media_tempo_caixa()+caixa2.media_tempo_caixa()+caixa3.media_tempo_caixa()+caixa4.media_tempo_caixa())/4)*tempo_medio(trasacao_workstation_service_date))/60,2),"minutos")
 
main_fila_com_peso()

fila-> [14, 11, 6, 15, 2, 14, 17, 15, 5, 10, 8, 3, 17, 5, 8, 19, 13, 4, 2, 14, 77, 64, 80, 41] tamanho da fila-> 24
fila-> [14, 5, 10, 4, 10, 14, 14, 9, 18, 10, 15, 11, 15, 9, 13, 19, 37, 48, 46, 28, 48, 32, 94] tamanho da fila-> 23
fila-> [6, 10, 17, 15, 6, 5, 8, 1, 12, 9, 9, 12, 9, 17, 17, 2, 6, 3, 3, 13, 11, 58, 80, 88, 95] tamanho da fila-> 25
fila-> [13, 10, 3, 3, 16, 17, 2, 4, 2, 14, 1, 16, 3, 5, 2, 7, 6, 5, 14, 16, 18, 19, 57, 74, 30, 56, 34, 50] tamanho da fila-> 28
inicia cliente ########################################## Cliente atual 0 QTD iten do cliente 14 Caixa 1 Tempo atual 2
inicia cliente ########################################## Cliente atual 0 QTD iten do cliente 14 Caixa 2 Tempo atual 2
inicia cliente ########################################## Cliente atual 0 QTD iten do cliente 6 Caixa 3 Tempo atual 2
inicia cliente ########################################## Cliente atual 0 QTD iten do cliente 13 Caixa 4 Tempo atual 2
o cliente Saiu################################

Caixa 1 media de tempo 2.97 minutos Cliente atendidos 24 Tempo de simulação caixa 1 71.5 minutos
Caixa 2 media de tempo 3.49 minutos Cliente atendidos 23 Tempo de simulação caixa 2 80.55 minutos
Caixa 3 media de tempo 3.14 minutos Cliente atendidos 25 Tempo de simulação caixa 3 78.86 minutos
Caixa 4 media de tempo 2.72 minutos Cliente atendidos 28 Tempo de simulação caixa 4 76.56 minutos
media de tempo dos caixas 3.08 minutos


### Simulação considerando filas com o mesmo tamanho

In [81]:
def main_filas_distribuidas_iguais():
  clientes=[]
  env = simpy.Environment()
  filas = criação_de_filas_linear()
  caixa1=caixa(1,env,filas[0])
  caixa2=caixa(2,env,filas[1])
  caixa3=caixa(3,env,filas[2])
  caixa4=caixa(4,env,filas[3])
  caixa1.set_caixas([caixa2,caixa2,caixa4])
  caixa2.set_caixas([caixa1,caixa3,caixa4])
  caixa3.set_caixas([caixa1,caixa2,caixa4])
  caixa4.set_caixas([caixa1,caixa2,caixa3])
  env.run()
  print("Caixa 1 media de tempo",round((caixa1.media_tempo_caixa()*tempo_medio(trasacao_workstation_service_date))/60,2),"minutos","Cliente atendidos",len(caixa1.clientes_finalizados),"Tempo de simulação caixa 1",round((caixa1.clientes_finalizados[-1].tempo_final*tempo_medio(trasacao_workstation_service_date))/60,2), "minutos")
  print("Caixa 2 media de tempo",round((caixa2.media_tempo_caixa()*tempo_medio(trasacao_workstation_service_date))/60,2),"minutos","Cliente atendidos",len(caixa2.clientes_finalizados),"Tempo de simulação caixa 2",round((caixa2.clientes_finalizados[-1].tempo_final*tempo_medio(trasacao_workstation_service_date))/60,2), "minutos")
  print("Caixa 3 media de tempo",round((caixa3.media_tempo_caixa()*tempo_medio(trasacao_workstation_service_date))/60,2),"minutos","Cliente atendidos",len(caixa3.clientes_finalizados),"Tempo de simulação caixa 3",round((caixa3.clientes_finalizados[-1].tempo_final*tempo_medio(trasacao_workstation_service_date))/60,2), "minutos")
  print("Caixa 4 media de tempo",round((caixa4.media_tempo_caixa()*tempo_medio(trasacao_workstation_service_date))/60,2),"minutos","Cliente atendidos",len(caixa4.clientes_finalizados),"Tempo de simulação caixa 4",round((caixa4.clientes_finalizados[-1].tempo_final*tempo_medio(trasacao_workstation_service_date))/60,2), "minutos")
  print("media de tempo dos caixas",round((((caixa1.media_tempo_caixa()+caixa2.media_tempo_caixa()+caixa3.media_tempo_caixa()+caixa4.media_tempo_caixa())/4)*tempo_medio(trasacao_workstation_service_date))/60,2),"minutos")
main_filas_distribuidas_iguais()

fila-> [14, 10, 17, 3, 10, 2, 14, 4, 14, 9, 5, 3, 2, 6, 5, 5, 19, 18, 13, 4, 58, 80, 28, 48, 95] tamanho da fila-> 25
fila-> [14, 10, 10, 4, 15, 14, 8, 1, 9, 1, 10, 5, 9, 17, 14, 8, 2, 3, 19, 2, 57, 74, 30, 56, 41] tamanho da fila-> 25
fila-> [6, 11, 3, 16, 14, 6, 17, 12, 15, 16, 12, 15, 7, 17, 15, 16, 6, 3, 19, 14, 77, 46, 88, 32, 94] tamanho da fila-> 25
fila-> [13, 5, 6, 15, 17, 5, 2, 2, 18, 9, 10, 8, 3, 11, 17, 9, 13, 13, 11, 37, 48, 64, 80, 34, 50] tamanho da fila-> 25
inicia cliente ########################################## Cliente atual 0 QTD iten do cliente 14 Caixa 1 Tempo atual 2
inicia cliente ########################################## Cliente atual 0 QTD iten do cliente 14 Caixa 2 Tempo atual 2
inicia cliente ########################################## Cliente atual 0 QTD iten do cliente 6 Caixa 3 Tempo atual 2
inicia cliente ########################################## Cliente atual 0 QTD iten do cliente 13 Caixa 4 Tempo atual 2
o cliente Saiu################################

Caixa 1 media de tempo 2.98 minutos Cliente atendidos 25 Tempo de simulação caixa 1 74.87 minutos
Caixa 2 media de tempo 3.11 minutos Cliente atendidos 26 Tempo de simulação caixa 2 82.7 minutos
Caixa 3 media de tempo 3.09 minutos Cliente atendidos 24 Tempo de simulação caixa 3 74.41 minutos
Caixa 4 media de tempo 3.07 minutos Cliente atendidos 25 Tempo de simulação caixa 4 77.02 minutos
media de tempo dos caixas 3.06 minutos


### simulação considerando filas com prioridade (Padrão) 

In [129]:
def main_filas_com_prioridade():
  clientes=[]
  env = simpy.Environment()
  filas = criação_de_filas_com_prioridade()
  caixa1=caixa(1,env,filas[0])
  caixa2=caixa(2,env,filas[1])
  caixa3=caixa(3,env,filas[2])
  caixa4=caixa(4,env,filas[3])
  caixa1.set_caixas([caixa2,caixa2,caixa4])
  caixa2.set_caixas([caixa1,caixa3,caixa4])
  caixa3.set_caixas([caixa1,caixa2,caixa4])
  caixa4.set_caixas([caixa1,caixa2,caixa3])
  env.run()
  print("Caixa 1 media de tempo",round((caixa1.media_tempo_caixa()*tempo_medio(trasacao_workstation_service_date))/60,2),"minutos","Cliente atendidos",len(caixa1.clientes_finalizados),"Tempo de simulação caixa 1",round((caixa1.clientes_finalizados[-1].tempo_final*tempo_medio(trasacao_workstation_service_date))/60,2), "minutos")
  print("Caixa 2 media de tempo",round((caixa2.media_tempo_caixa()*tempo_medio(trasacao_workstation_service_date))/60,2),"minutos","Cliente atendidos",len(caixa2.clientes_finalizados),"Tempo de simulação caixa 2",round((caixa2.clientes_finalizados[-1].tempo_final*tempo_medio(trasacao_workstation_service_date))/60,2), "minutos")
  print("Caixa 3 media de tempo",round((caixa3.media_tempo_caixa()*tempo_medio(trasacao_workstation_service_date))/60,2),"minutos","Cliente atendidos",len(caixa3.clientes_finalizados),"Tempo de simulação caixa 3",round((caixa3.clientes_finalizados[-1].tempo_final*tempo_medio(trasacao_workstation_service_date))/60,2), "minutos")
  print("Caixa 4 media de tempo",round((caixa4.media_tempo_caixa()*tempo_medio(trasacao_workstation_service_date))/60,2),"minutos","Cliente atendidos",len(caixa4.clientes_finalizados),"Tempo de simulação caixa 4",round((caixa4.clientes_finalizados[-1].tempo_final*tempo_medio(trasacao_workstation_service_date))/60,2), "minutos")
  tempo_medio_filas_prioridade_padrao = round((((caixa1.media_tempo_caixa()+caixa2.media_tempo_caixa()+caixa3.media_tempo_caixa()+caixa4.media_tempo_caixa())/4)*tempo_medio(trasacao_workstation_service_date))/60,2)
  print("media de tempo dos caixas",tempo_medio_filas_prioridade_padrao,"minutos")
  return(tempo_medio_filas_prioridade_padrao)
  
tempo_medio_filas_prioridade_padrao=main_filas_com_prioridade()

fila-> [6, 10, 10, 5, 10, 3, 6, 3, 4, 10, 2, 6, 5, 8, 2, 4, 1, 2, 9, 9, 1, 9, 5, 10, 10, 3, 5, 8, 2, 9, 7, 3, 6, 5, 5, 8, 9, 2, 6, 3, 3, 4, 2] tamanho da fila-> 43
fila-> [14, 14, 13, 11, 17, 16, 15, 15, 14, 17, 14, 14, 17, 12, 14, 15, 18, 16, 12, 15, 17, 17, 11, 14, 15, 17, 16, 19, 13, 18, 13, 13, 19, 19, 11, 14, 28, 30] tamanho da fila-> 38
fila-> [37, 58, 57, 48, 46, 48, 56, 32, 34, 41, 50] tamanho da fila-> 11
fila-> [77, 80, 74, 64, 88, 80, 95, 94] tamanho da fila-> 8
inicia cliente ########################################## Cliente atual 0 QTD iten do cliente 6 Caixa 1 Tempo atual 2
inicia cliente ########################################## Cliente atual 0 QTD iten do cliente 14 Caixa 2 Tempo atual 2
inicia cliente ########################################## Cliente atual 0 QTD iten do cliente 37 Caixa 3 Tempo atual 2
inicia cliente ########################################## Cliente atual 0 QTD iten do cliente 77 Caixa 4 Tempo atual 2
o cliente Saiu#################################

Caixa 1 media de tempo 1.16 minutos Cliente atendidos 51 Tempo de simulação caixa 1 71.96 minutos
Caixa 2 media de tempo 2.47 minutos Cliente atendidos 31 Tempo de simulação caixa 2 78.55 minutos
Caixa 3 media de tempo 7.69 minutos Cliente atendidos 11 Tempo de simulação caixa 3 86.38 minutos
Caixa 4 media de tempo 12.23 minutos Cliente atendidos 7 Tempo de simulação caixa 4 85.92 minutos
media de tempo dos caixas 5.89 minutos


In [126]:
def porcetagem_melhorar(tempo,tempo_medio_filas_prioridade_padrao):
    porcentagem_atual=(tempo*100)/tempo_medio_filas_prioridade_padrao
    porcentagem=100-porcentagem_atual
    return(porcentagem)
    

## simulação com embalador (desconsiderar break time)

In [127]:
def main_com_embalador():
  clientes=[]
  env = simpy.Environment()
  filas = criação_de_filas_com_prioridade()
  caixa1=caixa(1,env,filas[0])
  caixa2=caixa(2,env,filas[1])
  caixa3=caixa(3,env,filas[2])
  caixa4=caixa(4,env,filas[3])
  caixa1.set_caixas([caixa2,caixa2,caixa4])
  caixa2.set_caixas([caixa1,caixa3,caixa4])
  caixa3.set_caixas([caixa1,caixa2,caixa4])
  caixa4.set_caixas([caixa1,caixa2,caixa3])
  env.run()
  print("Caixa 1 media de tempo",round((caixa1.media_tempo_caixa()*tempo_medio(trasacao_workstation_service_date,trantime=True))/60,2),"minutos","Cliente atendidos",len(caixa1.clientes_finalizados),"Tempo de simulação caixa 1",round((caixa1.clientes_finalizados[-1].tempo_final*tempo_medio(trasacao_workstation_service_date,trantime=True))/60,2), "minutos")
  print("Caixa 2 media de tempo",round((caixa2.media_tempo_caixa()*tempo_medio(trasacao_workstation_service_date,trantime=True))/60,2),"minutos","Cliente atendidos",len(caixa2.clientes_finalizados),"Tempo de simulação caixa 2",round((caixa2.clientes_finalizados[-1].tempo_final*tempo_medio(trasacao_workstation_service_date,trantime=True))/60,2), "minutos")
  print("Caixa 3 media de tempo",round((caixa3.media_tempo_caixa()*tempo_medio(trasacao_workstation_service_date,trantime=True))/60,2),"minutos","Cliente atendidos",len(caixa3.clientes_finalizados),"Tempo de simulação caixa 3",round((caixa3.clientes_finalizados[-1].tempo_final*tempo_medio(trasacao_workstation_service_date,trantime=True))/60,2), "minutos")
  print("Caixa 4 media de tempo",round((caixa4.media_tempo_caixa()*tempo_medio(trasacao_workstation_service_date,trantime=True))/60,2),"minutos","Cliente atendidos",len(caixa4.clientes_finalizados),"Tempo de simulação caixa 4",round((caixa4.clientes_finalizados[-1].tempo_final*tempo_medio(trasacao_workstation_service_date,trantime=True))/60,2), "minutos")
  tempo_medio_filas_prioridade_embalador = round((((caixa1.media_tempo_caixa()+caixa2.media_tempo_caixa()+caixa3.media_tempo_caixa()+caixa4.media_tempo_caixa())/4)*tempo_medio(trasacao_workstation_service_date,trantime=True))/60,2)
  print("media de tempo dos caixas",tempo_medio_filas_prioridade_embalador,"minutos")
  print("a simulação com embalador é :",round(porcetagem_melhorar(tempo_medio_filas_prioridade_embalador,tempo_medio_filas_prioridade_padrao),2),"% mais eficiente em referencia a simulaçao utilizando a media do tempo total")
main_com_embalador()

fila-> [6, 10, 10, 5, 10, 3, 6, 3, 4, 10, 2, 6, 5, 8, 2, 4, 1, 2, 9, 9, 1, 9, 5, 10, 10, 3, 5, 8, 2, 9, 7, 3, 6, 5, 5, 8, 9, 2, 6, 3, 3, 4, 2] tamanho da fila-> 43
fila-> [14, 14, 13, 11, 17, 16, 15, 15, 14, 17, 14, 14, 17, 12, 14, 15, 18, 16, 12, 15, 17, 17, 11, 14, 15, 17, 16, 19, 13, 18, 13, 13, 19, 19, 11, 14, 28, 30] tamanho da fila-> 38
fila-> [37, 58, 57, 48, 46, 48, 56, 32, 34, 41, 50] tamanho da fila-> 11
fila-> [77, 80, 74, 64, 88, 80, 95, 94] tamanho da fila-> 8
inicia cliente ########################################## Cliente atual 0 QTD iten do cliente 6 Caixa 1 Tempo atual 2
inicia cliente ########################################## Cliente atual 0 QTD iten do cliente 14 Caixa 2 Tempo atual 2
inicia cliente ########################################## Cliente atual 0 QTD iten do cliente 37 Caixa 3 Tempo atual 2
inicia cliente ########################################## Cliente atual 0 QTD iten do cliente 77 Caixa 4 Tempo atual 2
o cliente Saiu#################################

Caixa 1 media de tempo 0.59 minutos Cliente atendidos 51 Tempo de simulação caixa 1 36.5 minutos
Caixa 2 media de tempo 1.26 minutos Cliente atendidos 31 Tempo de simulação caixa 2 39.85 minutos
Caixa 3 media de tempo 3.9 minutos Cliente atendidos 11 Tempo de simulação caixa 3 43.82 minutos
Caixa 4 media de tempo 6.2 minutos Cliente atendidos 7 Tempo de simulação caixa 4 43.58 minutos
media de tempo dos caixas 2.99 minutos
a simulação com embalador é : 49.24 % mais eficiente em referencia a simulaçao utilizando a media do tempo total


## Simulação com cartão 

In [128]:
def main_com_Cartão():
  clientes=[]
  env = simpy.Environment()
  filas = criação_de_filas_com_prioridade()
  caixa1=caixa(1,env,filas[0])
  caixa2=caixa(2,env,filas[1])
  caixa3=caixa(3,env,filas[2])
  caixa4=caixa(4,env,filas[3])
  caixa1.set_caixas([caixa2,caixa2,caixa4])
  caixa2.set_caixas([caixa1,caixa3,caixa4])
  caixa3.set_caixas([caixa1,caixa2,caixa4])
  caixa4.set_caixas([caixa1,caixa2,caixa3])
  env.run()
  print("Caixa 1 media de tempo",round((caixa1.media_tempo_caixa()*tempo_medio(trasacao_workstation_service_date_tncard))/60,2),"minutos","Cliente atendidos",len(caixa1.clientes_finalizados),"Tempo de simulação caixa 1",round((caixa1.clientes_finalizados[-1].tempo_final*tempo_medio(trasacao_workstation_service_date_tncard))/60,2), "minutos")
  print("Caixa 2 media de tempo",round((caixa2.media_tempo_caixa()*tempo_medio(trasacao_workstation_service_date_tncard))/60,2),"minutos","Cliente atendidos",len(caixa2.clientes_finalizados),"Tempo de simulação caixa 2",round((caixa2.clientes_finalizados[-1].tempo_final*tempo_medio(trasacao_workstation_service_date_tncard))/60,2), "minutos")
  print("Caixa 3 media de tempo",round((caixa3.media_tempo_caixa()*tempo_medio(trasacao_workstation_service_date_tncard))/60,2),"minutos","Cliente atendidos",len(caixa3.clientes_finalizados),"Tempo de simulação caixa 3",round((caixa3.clientes_finalizados[-1].tempo_final*tempo_medio(trasacao_workstation_service_date_tncard))/60,2), "minutos")
  print("Caixa 4 media de tempo",round((caixa4.media_tempo_caixa()*tempo_medio(trasacao_workstation_service_date_tncard))/60,2),"minutos","Cliente atendidos",len(caixa4.clientes_finalizados),"Tempo de simulação caixa 4",round((caixa4.clientes_finalizados[-1].tempo_final*tempo_medio(trasacao_workstation_service_date_tncard))/60,2), "minutos")
  tempo_medio_filas_prioridade_cartao = round((((caixa1.media_tempo_caixa()+caixa2.media_tempo_caixa()+caixa3.media_tempo_caixa()+caixa4.media_tempo_caixa())/4)*tempo_medio(trasacao_workstation_service_date_tncard))/60,2)
  print("a simulação com Cartão é :",round(porcetagem_melhorar(tempo_medio_filas_prioridade_cartao,tempo_medio_filas_prioridade_padrao),2),"% mais eficiente em referencia a simulaçao utilizando a media do tempo total")
main_com_Cartão()

fila-> [6, 10, 10, 5, 10, 3, 6, 3, 4, 10, 2, 6, 5, 8, 2, 4, 1, 2, 9, 9, 1, 9, 5, 10, 10, 3, 5, 8, 2, 9, 7, 3, 6, 5, 5, 8, 9, 2, 6, 3, 3, 4, 2] tamanho da fila-> 43
fila-> [14, 14, 13, 11, 17, 16, 15, 15, 14, 17, 14, 14, 17, 12, 14, 15, 18, 16, 12, 15, 17, 17, 11, 14, 15, 17, 16, 19, 13, 18, 13, 13, 19, 19, 11, 14, 28, 30] tamanho da fila-> 38
fila-> [37, 58, 57, 48, 46, 48, 56, 32, 34, 41, 50] tamanho da fila-> 11
fila-> [77, 80, 74, 64, 88, 80, 95, 94] tamanho da fila-> 8
inicia cliente ########################################## Cliente atual 0 QTD iten do cliente 6 Caixa 1 Tempo atual 2
inicia cliente ########################################## Cliente atual 0 QTD iten do cliente 14 Caixa 2 Tempo atual 2
inicia cliente ########################################## Cliente atual 0 QTD iten do cliente 37 Caixa 3 Tempo atual 2
inicia cliente ########################################## Cliente atual 0 QTD iten do cliente 77 Caixa 4 Tempo atual 2
o cliente Saiu#################################

Caixa 2 media de tempo 2.12 minutos Cliente atendidos 31 Tempo de simulação caixa 2 67.22 minutos
Caixa 3 media de tempo 6.58 minutos Cliente atendidos 11 Tempo de simulação caixa 3 73.92 minutos
Caixa 4 media de tempo 10.47 minutos Cliente atendidos 7 Tempo de simulação caixa 4 73.53 minutos
a simulação com Cartão é : 14.43 % mais eficiente em referencia a simulaçao utilizando a media do tempo total
